# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import gensim.downloader as api
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.model_selection import train_test_split

# Load

In [ ]:
# Load dataset
data = pd.read_csv('/content/drive/MyDrive/ML dataset /final_tmdb_dataset.csv')
# Load pre-trained Word2Vec model (Google's pre-trained model)
word2vec_model = api.load("word2vec-google-news-300")  # Pre-trained on Google News


# Word Embeddings

using word to vec and taking the Avg. to have a vector of size 300 for each data entry

In [ ]:
# Function to convert text to word2vec embeddings
def get_word2vec_embedding(text, model):
    words = text.split()  # Assuming the text is already preprocessed
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:  # If no word is found in the model
        return np.zeros(model.vector_size)  # Return a zero vector
    return np.mean(word_vectors, axis=0)  # Average the word vectors to represent the document


In [ ]:
# Fill missing values in the text features with an empty string
data['overview_cleaned'] = data['overview_cleaned'].fillna('')
data['keywords_cleaned'] = data['keywords_cleaned'].fillna('')
data['title_cleaned'] = data['title_cleaned'].fillna('')

# converting the text featuers to word embeddings (300-D vector)
data['overview_w2v'] = data['overview_cleaned'].apply(lambda x: get_word2vec_embedding(x, word2vec_model))
data['keywords_w2v'] = data['keywords_cleaned'].apply(lambda x: get_word2vec_embedding(x, word2vec_model))
data['title_w2v'] = data['title_cleaned'].apply(lambda x: get_word2vec_embedding(x, word2vec_model))

spliting the 300-D vector to have 300 new features for each of our text features

In [ ]:
# Split the 300-D word2vec embeddings into separate columns

# For 'overview_w2v' embeddings
data[[f'overview_w2v{i}' for i in range(300)]] = pd.DataFrame(data['overview_w2v'].tolist(), index=data.index)

# For 'keywords_w2v' embeddings
data[[f'keywords_w2v{i}' for i in range(300)]] = pd.DataFrame(data['keywords_w2v'].tolist(), index=data.index)

# For 'title_w2v' embeddings
data[[f'title_w2v{i}' for i in range(300)]] = pd.DataFrame(data['title_w2v'].tolist(), index=data.index)


<ipython-input-5-eea311cc26cb>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[[f'overview_w2v{i}' for i in range(300)]] = pd.DataFrame(data['overview_w2v'].tolist(), index=data.index)
<ipython-input-5-eea311cc26cb>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[[f'overview_w2v{i}' for i in range(300)]] = pd.DataFrame(data['overview_w2v'].tolist(), index=data.index)
<ipython-input-5-eea311cc26cb>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

In [ ]:
data.drop(['overview_w2v', 'keywords_w2v', 'title_w2v'], axis=1, inplace=True)
data.drop(columns=['overview_cleaned', 'keywords_cleaned', 'title_cleaned'], inplace=True)

# 80% - 20% Data Split

In [ ]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Save the 80% training DataFrame as 'batates.csv'
train_df.to_csv('/content/drive/MyDrive/ML dataset /train.csv', index=False)

# Save the 20% testing DataFrame as 'homos.csv'
test_df.to_csv('/content/drive/MyDrive/ML dataset /test.csv', index=False)

print("DataFrames saved successfully.")

DataFrames saved successfully.


In [ ]:
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 32658 entries, 58301 to 76576
Columns: 1534 entries, vote_average to title_w2v299
dtypes: float64(1531), int64(3)
memory usage: 382.5 MB
None


In [ ]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 130629 entries, 113256 to 121958
Columns: 1534 entries, vote_average to title_w2v299
dtypes: float64(1531), int64(3)
memory usage: 1.5 GB
None
